In [34]:
import pandas as pd
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from datetime import datetime, timedelta
import io
import os
import ipywidgets as widgets
from dateutil.relativedelta import relativedelta
import numpy as np

class ExpenseManager:
    def __init__(self):
        self.expenses = pd.DataFrame()
        self.categories = []
        self.expenses_file = f'expenses_data_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'

    def add_expenses(self, expenses):
        expenses.columns = ['Date', 'Category', 'Amount']
        expenses['Date'] = pd.to_datetime(expenses['Date'])
        self.expenses = pd.concat([self.expenses, expenses], ignore_index=True)
        self.expenses.reset_index(drop=True, inplace=True)
        self.categories = sorted(list(set(self.expenses['Category'])))

    def append_selected_expenses(self, selected_expenses):
        if not os.path.exists(self.expenses_file):
            selected_expenses.to_csv(self.expenses_file, index=False, encoding='utf-8-sig', mode='w', header=['Date', 'Category', 'Amount'])
        else:
            selected_expenses.to_csv(self.expenses_file, index=False, encoding='utf-8-sig', mode='a', header=False)

    def select_expenses(self, period, category, amount):
        min_date = self.expenses['Date'].min()

        if period == 'Day':
            end_date = min_date + relativedelta(days=1)
        elif period == 'Week':
            end_date = min_date + relativedelta(weeks=1)
        elif period == 'Month':
            end_date = min_date + relativedelta(months=1)
        elif period == 'Year':
            end_date = min_date + relativedelta(years=1)
        elif period == '5 Years':
            end_date = min_date + relativedelta(years=5)
        elif period == '10 Years':
            end_date = min_date + relativedelta(years=10)
        else:
            end_date = datetime.now()

        selected_expenses = self.expenses[(self.expenses['Date'] >= min_date) &
                                           (self.expenses['Date'] < end_date) &
                                           (self.expenses['Category'] == category)].head(amount)
        return selected_expenses

    def load_expenses(self):
        try:
            return pd.read_csv(self.expenses_file, encoding='utf-8')
        except FileNotFoundError:
            print("No expenses data file found.")
            return pd.DataFrame()  # Return an empty DataFrame if file not found

    def generate_pdf_report(self, filename):
        png_folder = 'graph_pngs'
        png_files = [f for f in os.listdir(png_folder) if f.endswith('.png')]
        num_pngs = len(png_files)
        graphs_per_page = 3
        num_pages = -(-num_pngs // graphs_per_page)  

        c = canvas.Canvas(filename, pagesize=letter)
        for page_num in range(num_pages):
            c.drawString(100, 750, "Expense Report")
            c.drawString(100, 730, datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
            current_row = 0

            for i in range(graphs_per_page):
                graph_index = page_num * graphs_per_page + i
                if graph_index < num_pngs:
                    png_file = png_files[graph_index]
                    width, height = 200, 150  
                    x_offset = (c._pagesize[0] - width) / 2 
                    y_offset = 550 - current_row * 200  
                    c.drawImage(os.path.join(png_folder, png_file), x_offset, y_offset, width=width, height=height)
                    current_row += 1

            c.showPage()

        c.save()





class ExpenseInterface:
    def __init__(self, expense_manager):
        self.expense_manager = expense_manager
        self.load_csv_button = widgets.FileUpload(description="Upload CSV", style={'button_color': 'lightblue'})
        self.period_dropdown = widgets.Dropdown(description="Period:",
                                                options=['Day', 'Week', 'Month', 'Year', '5 Years', '10 Years', 'All Time'], style={'button_color': 'lightgreen'})
        self.category_dropdown = widgets.Dropdown(description="Category:", options=self.expense_manager.categories, style={'button_color': 'lightyellow'})
        self.amount_text = widgets.IntText(description="Amount:", style={'button_color': 'lightpink'})
        self.select_button = widgets.Button(description="Add exppense", style={'button_color': 'lightblue'})
        self.generate_pdf_button = widgets.Button(description="Generate PDF", style={'button_color': 'lightgreen'})
        self.output = widgets.Output()
        self.graph_dropdown = widgets.Dropdown(description="Graphs:",
                                               options=['Monthly Expense Breakdown',
                                                        'Expense Category Pie Chart',
                                                        'Trend Line Graph',
                                                        'Stacked Area Chart',
                                                        'Histogram of Expenses',
                                                        'Scatter Plot of Expenses by Date',
                                                        'Expense Category Bar Plot',
                                                        'Expenses Over Time Line Plot'], style={'button_color': 'lightyellow'})
        self.generate_png_button = widgets.Button(description="Generate PNG", style={'button_color': 'lightgreen'})

        self.select_button.on_click(self.select_expenses)
        self.generate_pdf_button.on_click(self.generate_pdf)
        self.load_csv_button.observe(self.load_csv, names='value')
        self.generate_png_button.on_click(self.generate_png)

        self.interface = widgets.VBox([self.load_csv_button, self.period_dropdown,
                                       self.category_dropdown, self.amount_text,
                                       self.select_button, self.graph_dropdown,
                                       self.generate_png_button, self.generate_pdf_button,
                                       self.output])
    def load_csv(self, change):
        for filename in change['new']:
            content = change['new'][filename]['content']
            csv_data = pd.read_csv(io.BytesIO(content), encoding='utf-8')
            self.expense_manager.add_expenses(csv_data)
            self.category_dropdown.options = self.expense_manager.categories

    def select_expenses(self, b):
        period = self.period_dropdown.value
        category = self.category_dropdown.value
        amount = self.amount_text.value

        selected_expenses = self.expense_manager.select_expenses(period, category, amount)
        with self.output:
            print(selected_expenses)
            self.expense_manager.append_selected_expenses(selected_expenses)

    def generate_pdf(self, b):
        self.expense_manager.generate_pdf_report("expense_report.pdf")

    def generate_png(self, b):
        selected_graph = self.graph_dropdown.value
        try:
            if selected_graph == 'Monthly Expense Breakdown':
                self.generate_monthly_expense_breakdown()
            elif selected_graph == 'Expense Category Pie Chart':
                self.generate_expense_category_pie_chart()
            elif selected_graph == 'Trend Line Graph':
                self.generate_trend_line_graph()
            elif selected_graph == 'Stacked Area Chart':
                self.generate_stacked_area_chart()
            elif selected_graph == 'Histogram of Expenses':
                self.generate_expense_histogram()
            elif selected_graph == 'Scatter Plot of Expenses by Date':
                self.generate_expense_scatter_plot()
            elif selected_graph == 'Expense Category Bar Plot':
                self.generate_expense_category_bar_plot()
            elif selected_graph == 'Expenses Over Time Line Plot':
                self.generate_expenses_over_time_line_plot()
        except ValueError as e:
            with self.output:
                print("Error:", e)

    def generate_monthly_expense_breakdown(self):
        plt.figure(figsize=(8, 6), dpi=200)  # Adjust figsize and dpi for higher resolution
        expenses = self.expense_manager.load_expenses()
        expenses['Date'] = pd.to_datetime(expenses['Date'])  # Ensure 'Date' column is datetime type
        expenses.set_index('Date', inplace=True)  # Set 'Date' as index
        expenses.resample('M')['Amount'].sum().plot(kind='bar', color='skyblue')
        plt.title('Monthly Expense Breakdown')
        plt.xlabel('Month')
        plt.ylabel('Total Expenses')
        plt.xticks(rotation=45)
        plt.tight_layout()
        self.save_png('monthly_expense_breakdown.png')

    def generate_expense_category_pie_chart(self):
        plt.figure(figsize=(8, 6), dpi=200)  # Adjust figsize and dpi for higher resolution
        expenses = self.expense_manager.load_expenses()
        expenses['Date'] = pd.to_datetime(expenses['Date'])  # Ensure 'Date' column is datetime type
        expenses.set_index('Date', inplace=True)  # Set 'Date' as index
        expenses.groupby('Category')['Amount'].sum().plot(kind='pie', autopct='%1.1f%%')
        plt.title('Expense Category Breakdown')
        plt.ylabel('')
        plt.tight_layout()
        self.save_png('expense_category_pie_chart.png')

    def generate_trend_line_graph(self):
        plt.figure(figsize=(8, 6), dpi=200)  # Adjust figsize and dpi for higher resolution
        expenses = self.expense_manager.load_expenses()
        expenses['Date'] = pd.to_datetime(expenses['Date'])  # Ensure 'Date' column is datetime type
        expenses.set_index('Date', inplace=True)  # Set 'Date' as index
        expenses.resample('M')['Amount'].sum().plot(kind='line', marker='o', color='green')
        plt.title('Expense Trend Over Time')
        plt.xlabel('Date')
        plt.ylabel('Total Expenses')
        plt.xticks(rotation=45)
        plt.tight_layout()
        self.save_png('trend_line_graph.png')

    def generate_stacked_area_chart(self):
        plt.figure(figsize=(8, 6), dpi=200)  # Adjust figsize and dpi for higher resolution
        expenses = self.expense_manager.load_expenses()
        expenses['Date'] = pd.to_datetime(expenses['Date'])  # Ensure 'Date' column is datetime type
        expenses.set_index('Date', inplace=True)  # Set 'Date' as index
        expenses.groupby(['Date', 'Category'])['Amount'].sum().unstack().plot.area()
        plt.title('Stacked Area Chart of Expenses Over Time')
        plt.xlabel('Date')
        plt.ylabel('Total Expenses')
        plt.xticks(rotation=45)
        plt.tight_layout()
        self.save_png('stacked_area_chart.png')

    def generate_expense_histogram(self):
        plt.figure(figsize=(8, 6), dpi=200)  # Adjust figsize and dpi for higher resolution
        expenses = self.expense_manager.load_expenses()
        expenses['Date'] = pd.to_datetime(expenses['Date'])  # Ensure 'Date' column is datetime type
        expenses.set_index('Date', inplace=True)  # Set 'Date' as index
        expenses.hist(column='Amount', by='Category', bins=10, figsize=(10, 6), sharex=True, sharey=True)
        plt.suptitle('Histogram of Expenses by Category')
        plt.tight_layout()
        self.save_png('expense_histogram.png')

    def generate_expense_scatter_plot(self):
        plt.figure(figsize=(8, 6), dpi=200)  # Adjust figsize and dpi for higher resolution
        expenses = self.expense_manager.load_expenses()
        expenses['Date'] = pd.to_datetime(expenses['Date'])  # Ensure 'Date' column is datetime type
        unique_categories = expenses['Category'].unique()
        for category in unique_categories:
            category_expenses = expenses[expenses['Category'] == category]
            plt.scatter(category_expenses.index, category_expenses['Amount'], label=category)
        plt.title('Scatter Plot of Expenses by Date')
        plt.xlabel('Date')
        plt.ylabel('Amount')
        plt.legend()
        plt.tight_layout()
        self.save_png('expense_scatter_plot.png')
        
    def generate_expense_category_bar_plot(self):
        plt.figure(figsize=(8, 6), dpi=200)
        expenses = self.expense_manager.load_expenses()
        expenses['Date'] = pd.to_datetime(expenses['Date'])
        expenses.set_index('Date', inplace=True)
        # Define a color palette for the bar plot
        colors = plt.cm.tab10(np.arange(len(expenses['Category'].unique())))
        expenses.groupby('Category')['Amount'].sum().plot(kind='bar', color=colors)
        plt.title('Total Expenses per Category')
        plt.xlabel('Category')
        plt.ylabel('Total Expenses')
        plt.xticks(rotation=45)
        plt.tight_layout()
        self.save_png('expense_category_bar_plot.png')  
        
    def generate_expenses_over_time_line_plot(self):
        plt.figure(figsize=(8, 6), dpi=200)
        expenses = self.expense_manager.load_expenses()
        expenses['Date'] = pd.to_datetime(expenses['Date'])
        expenses.set_index('Date', inplace=True)
        unique_categories = expenses['Category'].unique()
        # Define a color palette for the line plot
        colors = plt.cm.tab10(np.linspace(0, 1, len(unique_categories)))
        for i, category in enumerate(unique_categories):
            category_expenses = expenses[expenses['Category'] == category]
            plt.plot(category_expenses.index, category_expenses['Amount'], label=category, color=colors[i])
        plt.title('Expenses Over Time')
        plt.xlabel('Date')
        plt.ylabel('Total Expenses')
        plt.xticks(rotation=45)
        plt.legend()
        plt.tight_layout()
        self.save_png('expenses_over_time_line_plot.png')

    def save_png(self, filename):
        if not os.path.exists('graph_pngs'):
            os.makedirs('graph_pngs')
        filepath = os.path.join('graph_pngs', filename)
        plt.savefig(filepath, format='png')
        plt.close()


expense_manager = ExpenseManager()
expense_interface = ExpenseInterface(expense_manager)
widgets.VBox([expense_interface.interface])